# 12장

In [1]:
from tensorflow import keras
import tensorflow as tf

In [2]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.,]])
print(t,
      t[:, :, tf.newaxis],
      t+10,
      tf.square(t),
      t@tf.transpose(t),
      tf.cast(t, tf.float16),
      sep='\n\n')

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)

tf.Tensor(
[[[1.]
  [2.]
  [3.]]

 [[4.]
  [5.]
  [6.]]], shape=(2, 3, 1), dtype=float32)

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)

tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)

tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float16)


In [3]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) -0.5

    return tf.where(is_small_error, squared_loss, linear_loss)

In [4]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [5]:
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 2s 3ms/step - loss: 0.5780 - mae: 0.9432 - val_loss: 0.3113 - val_mae: 0.6192
Epoch 2/2
363/363 [==============================] - 1s 3ms/step - loss: 0.2185 - mae: 0.5178 - val_loss: 0.2455 - val_mae: 0.5346


In [6]:
model.save("my_model_with_a_custom_loss.h5")
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [7]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 3ms/step - loss: 0.2051 - mae: 0.4963 - val_loss: 0.2021 - val_mae: 0.4898
Epoch 2/2
363/363 [==============================] - 1s 3ms/step - loss: 0.1983 - mae: 0.4879 - val_loss: 0.2070 - val_mae: 0.4905


In [8]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer='nadam', metrics=['mae'])

In [9]:
model.save("my_model_with_a_custom_loss.h5")

In [10]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [11]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 3ms/step - loss: 0.2203 - mae: 0.4875 - val_loss: 0.2000 - val_mae: 0.4639
Epoch 2/2
363/363 [==============================] - 1s 3ms/step - loss: 0.2166 - mae: 0.4828 - val_loss: 0.2319 - val_mae: 0.4833


In [14]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
    
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        
        return {**base_config, 'threshold': self.threshold}

model.compile(loss=HuberLoss(2.0), optimizer='nadam', metrics=['mae'])
model.save("my_model_with_a_custom_loss.h5")
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"HuberLoss": HuberLoss})

In [15]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 2s 5ms/step - loss: 0.2126 - mae: 0.4789 - val_loss: 0.2081 - val_mae: 0.4631
Epoch 2/2
363/363 [==============================] - 1s 3ms/step - loss: 0.2092 - mae: 0.4737 - val_loss: 0.2040 - val_mae: 0.4654


In [18]:
def  my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorit_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [20]:
layer = keras.layers.Dense(30, activation=my_softplus,
                           kernel_initializer=my_glorit_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

In [21]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    # 부모 클래스에 생성자와 get_config가 정의돼 있지 않아 호출(super) 불요.
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, weights):
        # loss, layer, model의 경우 call / regularizer, initializer, constraint의 경우 __call__
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {'factor': self.factor}